In [44]:
using Gmsh
using Gridap
using GridapGmsh
using LinearAlgebra

Import the Mesh to a model and get the nodes and elements

In [4]:
model = GmshDiscreteModel("../data/VenturaAccelerometer/test_mesh_v2.msh")
nodes = model.grid.node_coordinates



Info    : Reading '../data/VenturaAccelerometer/test_mesh_v2.msh'...
Info    : 1 entity
Info    : 8 nodes
Info    : 6 elements
Info    : Done reading '../data/VenturaAccelerometer/test_mesh_v2.msh'


8-element Vector{VectorValue{2, Float64}}:
 VectorValue{2, Float64}(-1.0, 0.5)
  VectorValue{2, Float64}(0.0, 1.0)
 VectorValue{2, Float64}(-1.0, 1.0)
 VectorValue{2, Float64}(-1.0, 0.0)
  VectorValue{2, Float64}(0.0, 0.0)
  VectorValue{2, Float64}(1.0, 0.0)
  VectorValue{2, Float64}(1.0, 0.5)
  VectorValue{2, Float64}(1.0, 1.0)

In [5]:
elements = model.grid.cell_node_ids

6-element Gridap.Arrays.Table{Int32, Vector{Int32}, Vector{Int32}}:
 [1, 2, 3]
 [1, 4, 5]
 [5, 6, 7]
 [1, 2, 5]
 [2, 5, 7]
 [2, 7, 8]

Get all the areas of the elements

In [6]:
Areas = zeros(length(elements))
A = 0
for (i, element) in enumerate(elements)
    area = 0
    for j in 1:length(element)
        k = j % length(element) + 1
        area += nodes[element[j]][1] * nodes[element[k]][2] - nodes[element[j]][2] * nodes[element[k]][1]
    end 
    Areas[i] = abs(area)/2  

end
println(Areas)
println(sum(Areas))
println(length)


[0.25, 0.25, 0.25, 0.5, 0.5, 0.25]
2.0


length

Define the F funtion

In [61]:
using Plots
function f(x, y, t)
    x0 = 0
    y0 = 0.5
    sigmax = 0.2
    sigmay = 0.2
    return exp(-(((x-x0)^2)/(2*sigmax^2) + (y-y0)^2/(2*sigmay^2)))
end


f (generic function with 1 method)

Compute the K matrix FINALLY

In [63]:
N = length(nodes)
Ke = zeros(3,3)
K = zeros(N,N)
for (index,element) in enumerate(elements)
    ABC = []
    for nd in element
        a , b= nodes[nd]
        ABC = vcat(ABC, [a, b, 1])
        
        
    end
    ABC = transpose(reshape(ABC, 3, 3))
    ABC = inv(float.(ABC))

    for i in 1:3
        for j in 1:3
            Ke[i,j] = ABC[1,i]*ABC[1,j] + ABC[2, i]*ABC[2, j]
        end
    end

    Ke = Ke * Areas[index]

    for (local_i, real_i) in enumerate(element)
        for (local_j, real_j) in enumerate(element)
            K[real_i, real_j]+= Ke[local_i, local_j]
        end
    end
    

end

display(K)

8×8 Matrix{Float64}:
  2.5   -0.25  -1.0   -1.0   -0.25   0.0    0.0    0.0
 -0.25   1.75  -0.25   0.0   -0.75   0.0   -0.25  -0.25
 -1.0   -0.25   1.25   0.0    0.0    0.0    0.0    0.0
 -1.0    0.0    0.0    1.25  -0.25   0.0    0.0    0.0
 -0.25  -0.75   0.0   -0.25   1.75  -0.25  -0.25   0.0
  0.0    0.0    0.0    0.0   -0.25   1.25  -1.0    0.0
  0.0   -0.25   0.0    0.0   -0.25  -1.0    2.5   -1.0
  0.0   -0.25   0.0    0.0    0.0    0.0   -1.0    1.25

Create de M matrix

In [78]:
M = zeros(N,N)

for (index,element) in enumerate(elements)
    Me = I(3)
    Me = Me*Areas[index]/3
    #display(Me)
    for (local_i, real_i) in enumerate(element)
        M[real_i, real_i]+= Me[local_i, local_i]
    end
end
display(M)


8×8 Matrix{Float64}:
 0.333333  0.0  0.0        0.0        0.0  0.0        0.0       0.0
 0.0       0.5  0.0        0.0        0.0  0.0        0.0       0.0
 0.0       0.0  0.0833333  0.0        0.0  0.0        0.0       0.0
 0.0       0.0  0.0        0.0833333  0.0  0.0        0.0       0.0
 0.0       0.0  0.0        0.0        0.5  0.0        0.0       0.0
 0.0       0.0  0.0        0.0        0.0  0.0833333  0.0       0.0
 0.0       0.0  0.0        0.0        0.0  0.0        0.333333  0.0
 0.0       0.0  0.0        0.0        0.0  0.0        0.0       0.0833333

Damping Matrix

In [81]:
α = 0.1
C = M * α
display(C)

8×8 Matrix{Float64}:
 0.0333333  0.0   0.0         …  0.0         0.0        0.0
 0.0        0.05  0.0            0.0         0.0        0.0
 0.0        0.0   0.00833333     0.0         0.0        0.0
 0.0        0.0   0.0            0.0         0.0        0.0
 0.0        0.0   0.0            0.0         0.0        0.0
 0.0        0.0   0.0         …  0.00833333  0.0        0.0
 0.0        0.0   0.0            0.0         0.0333333  0.0
 0.0        0.0   0.0            0.0         0.0        0.00833333